In [17]:
!pip install --upgrade gradio sacremoses datasets pyarrow --quiet

In [ ]:
from transformers import Wav2Vec2Model
from transformers import pipeline
import gradio as gr
import json
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor

In [10]:
vocab_dict = {
    ' ': 0,
    'r': 1,
    'f': 2,
    'n': 3,
    '3': 4,
    'l': 5,
    'q': 6,
    'j': 7,
    'u': 8,
    'd': 9,
    'o': 10,
    '2': 11,
    'y': 12,
    'g': 13,
    'k': 14,
    'p': 15,
    'i': 16,
    'x': 17,
    'z': 18,
    'w': 19,
    'b': 20,
    "'": 21,
    'h': 22,
    'c': 23,
    'v': 24,
    '4': 25,
    's': 26,
    'm': 27,
    't': 28,
    '0': 29,
    'a': 30,
    'e': 31
}
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [24]:


feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1,
    sampling_rate=16000,
    padding_value=0.0,
    do_normalize=True,
    return_attention_mask=True
)
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
from transformers import Wav2Vec2ForCTC
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor,
    tokenizer=tokenizer
)
model = Wav2Vec2ForCTC.from_pretrained(
    "Shawal777/nyankole_wav2vec2-runpod-xls-r-300m-lrPlateau",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
    ignore_mismatched_sizes=True
)
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer = tokenizer,
    feature_extractor=feature_extractor,
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Shawal777/nyankole_wav2vec2-runpod-xls-r-300m-lrPlateau and are newly initialized because the shapes did not match:
- lm_head.bias: found shape torch.Size([32]) in the checkpoint and torch.Size([36]) in the model instantiated
- lm_head.weight: found shape torch.Size([32, 1024]) in the checkpoint and torch.Size([36, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
demo = gr.Interface.from_pipeline(pipe)
demo.launch(
    debug=True
)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://ff7a61ff7e4201ff1a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
